# DataTalks MLOps Week 1 Homework

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import pickle 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

## Q1. Downloading the data


Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [8]:
!wget -nv https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -O ./data/yellow_tripdata_2023-01.parquet
!wget -nv https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -O ./data/yellow_tripdata_2023-02.parquet

2025-05-19 16:55:11 URL:https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet [47673370/47673370] -> "./data/yellow_tripdata_2023-01.parquet" [1]
2025-05-19 16:56:41 URL:https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet [47748012/47748012] -> "./data/yellow_tripdata_2023-02.parquet" [1]


In [9]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

From the info above, we observe there are 19 columns.

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [10]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df.duration = df.duration.dt.total_seconds() / 60
print(f"{df.duration.std()=: .2f}")

df.duration.std()= 42.59


We observe that the standard deviation of trips duration in Jan is 42.59 minutes.

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [11]:
before = df.shape[0]
print(f"Number of records before dropping: {before}")
df = df[(df.duration >= 1) & (df.duration <= 60)]
after = df.shape[0]
print(f"Number of records after dropping: {after}")
print(f"{after*100/before:.0f}%")

Number of records before dropping: 3066766
Number of records after dropping: 3009173
98%


We see that after dropping outliers, we still have 98% of original records.

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [12]:
categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']
df[categorical] = df[categorical].astype(str)

In [13]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)
print(X_train.shape)

(3009173, 515)


We observe that there are 515 columns.

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [14]:
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')
df_val['duration'] = df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']
df_val.duration = df_val.duration.dt.total_seconds() / 60
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
df_val[categorical] = df_val[categorical].astype(str)

In [15]:
df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [16]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [17]:
target = 'duration'
y_train = df[target].values
y_val = df_val[target].values

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
root_mean_squared_error(y_train, y_pred)

7.649261931416412

We observe that the answer 7.64 is closest to the produced RMSE.

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [19]:
y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

7.8118162035401735

We observe that 7.81 is closest to the produced answer.